In [19]:
# Загрузим нужные библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import plt, mpl
import seaborn as sns
mpl.rcParams['font.family'] = 'serif'
import math
%matplotlib inline
plt.style.use('fivethirtyeight')
# Чистка отклонений
import scipy.stats as stats
# Уберем warnings
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
# Тематическое моделирование
import gensim

from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [2]:
# Тематическое моделирование
#test_df = them_topic(test_df, features='features')
#train_df = them_topic(train_df, features='features')



train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')




# Чистка широты и долготы
train_df["longitude"]=train_df["longitude"].apply(lambda x:-73.75 if x>=-73.75 else x)
test_df["longitude"]=test_df["longitude"].apply(lambda x:-74.05 if x<=-74.05 else x)
train_df["latitude"]=train_df["latitude"].apply(lambda x:40.4 if x<=40.4 else x)
test_df["latitude"]=test_df["latitude"].apply(lambda x:40.9 if x>=40.9 else x)

mean_price = int(train_df['price'].mean())
test_df.loc[test_df['price']<200,'price'] = mean_price
train_df.loc[train_df['price']<200,'price'] = mean_price
# Метапризнак предсказ цены на основе координат
#X_train_coor = train_df[['latitude', 'longitude']]
#X_test_coor = test_df[['latitude', 'longitude']]

#y_train_price = train_df.price
#y_test_price = test_df.price

#neigh_train = KNeighborsRegressor(n_neighbors=2)
#neigh_train.fit(X_train_coor, y_train_price)

#neigh_test = KNeighborsRegressor(n_neighbors=2)
#neigh_test.fit(X_test_coor, y_test_price)

#train_df['price_comparison'] = train_df['price'] /  neigh_train.predict(X_train_coor)
#test_df['price_comparison'] = test_df['price'] /  neigh_test.predict(X_test_coor)


# Функция расчета расстояния от манхэттэна
def distance_pair(lat1, lon1, lat2, lon2):
    # Перевод в радианы
    p = 0.017453292519943295     #Pi/180
    haver_formula = 0.5 - np.cos((lat2 - lat1) * p)/2 +  np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    d_2_point = 6367 *2 * np.arcsin(np.sqrt(haver_formula)) 
    return d_2_point
distance_pairs = np.vectorize(distance_pair)

# Расстояние от финансового центра
train_df['distance_manh'] = distance_pairs(train_df.latitude, train_df.longitude, 40.70858733058446, -74.01098113951349)
test_df['distance_manh'] = distance_pairs(test_df.latitude, test_df.longitude, 40.70858733058446, -74.01098113951349)


# Расстояние от центр парка
train_df['distance_central_park'] = distance_pairs(train_df.latitude, train_df.longitude, 40.77897128595648, -73.96656147253005)
test_df['distance_central_park'] = distance_pairs(test_df.latitude, test_df.longitude, 40.77897128595648, -73.96656147253005)

# Расстояние от  Бронкс
train_df['bronx'] = distance_pairs(train_df.latitude, train_df.longitude, 40.84928609864644, -73.88844362698887)
test_df['bronx'] = distance_pairs(test_df.latitude, test_df.longitude, 40.84928609864644, -73.88844362698887)

# Расстояние от Район Queens

train_df['queens'] = distance_pairs(train_df.latitude, train_df.longitude, 40.747803602867066, -73.90170458998642)
test_df['queens'] = distance_pairs(test_df.latitude, test_df.longitude, 40.747803602867066, -73.90170458998642)

# Расстояние от Район Brooklin кладбище

train_df['brooklin_cometery'] = distance_pairs(train_df.latitude, train_df.longitude, 40.68822862016868, -73.87615328604328)
test_df['brooklin_cometery'] = distance_pairs(test_df.latitude, test_df.longitude, 40.68822862016868, -73.87615328604328)

# Расстояние от Район Brooklin
train_df['brooklin'] = distance_pairs(train_df.latitude, train_df.longitude, 40.62506864788317, -73.96786745767038)
test_df['brooklin'] = distance_pairs(test_df.latitude, test_df.longitude, 40.62506864788317, -73.96786745767038)






# Кластеризация координат
geo = ['latitude', 'longitude']
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=42)

# Метки кластеров внесем в переменную clusters
train_df['clusters'] = kmeans.fit(train_df[geo]).labels_
test_df['clusters'] = kmeans.fit(test_df[geo]).labels_


#test_df = test.copy()


train_df['num_photos'] = train_df['photos'].apply(len)
test_df['num_photos'] = test_df['photos'].apply(len)

# Число признаков
train_df['num_features'] = train_df['features'].apply(len)
test_df['num_features'] = test_df['features'].apply(len)

# Количество слов
train_df['num_description_words'] = train_df['description'].apply(lambda x: len(x.split(" ")))
test_df['num_description_words'] = test_df['description'].apply(lambda x: len(x.split(" ")))


# Даты
train_df['created'] = pd.to_datetime(train_df['created'])
test_df['created'] = pd.to_datetime(test_df['created'])

# Извлекаем время
train_df['created_year'] = train_df['created'].dt.year
test_df['created_year'] = test_df['created'].dt.year
train_df['created_month'] = train_df['created'].dt.month
test_df['created_month'] = test_df['created'].dt.month
train_df['created_day'] = train_df['created'].dt.day
test_df['created_day'] = test_df['created'].dt.day
train_df['created_hour'] = train_df['created'].dt.hour
test_df['created_hour'] = test_df['created'].dt.hour



# Фичи по ванной и спальням
# Разл в комнатах
train_df['room_difference'] = train_df['bedrooms'] - train_df['bathrooms']
test_df['room_difference'] = test_df['bedrooms'] - test_df['bathrooms']
# Общее количество
train_df['total_rooms'] = train_df['bedrooms'] + train_df['bathrooms']
test_df['total_rooms'] = test_df['bedrooms'] + test_df['bathrooms']
# Цена 1 добавляется чтобы исключить деление на ноль
train_df['price_per_room'] = train_df['price'] / (train_df['total_rooms'] + 1)
test_df['price_per_room'] = test_df['price'] / (test_df['total_rooms'] + 1)
# Цена за спальню
train_df['price_per_bedroom'] =train_df['price']/(train_df['bedrooms'] + 1)
test_df['price_per_bedroom'] =test_df['price']/(test_df['bedrooms'] +1)

# Цена за ванную
train_df['price_per_bathroom'] =train_df['price']/(train_df['bathrooms'] +1)
test_df['price_per_bathroom'] =test_df['price']/(test_df['bathrooms'] +1)

# Близость к метро
subway = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv') # сторонние данные
subway = subway[['Station Name', 'Station Latitude', 'Station Longitude']]
subway = subway.groupby(['Station Name']).mean().reset_index(drop=True)


# Функция минимального расстояния от метро
def get_subway_distance(location):
    distances = distance_pairs(location[0], location[1], subway['Station Latitude'], subway['Station Longitude'])    
    return min(distances)

# Расчет минимальной дистанции до метро
train_df['location'] = train_df[['latitude', 'longitude']].values.tolist()
test_df['location'] = test_df[['latitude', 'longitude']].values.tolist()

train_df['subway_distance'] = train_df['location'].apply(get_subway_distance)
test_df['subway_distance'] = test_df['location'].apply(get_subway_distance)


# Плотность по координатам https://www.kaggle.com/code/maheshak04/rh004
train_df["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test_df["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train_df['pos'].value_counts()
dvals = vals.to_dict()
train_df["density"] = train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
test_df["density"] = test_df['pos'].apply(lambda x: dvals.get(x, vals.min()))



# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["listing_id1"] = train_df["listing_id"] - 68119576.0
test_df["listing_id1"] =  test_df["listing_id"] - 68119576.0

train_df["total_days"] =   (train_df["created_month"] -4.0)*30 + train_df["created_day"] +  train_df["created_hour"] /25.0
test_df["total_days"] =(test_df["created_month"] -4.0)*30 + test_df["created_day"] +  test_df["created_hour"] /25.0        
train_df["diff_rank"]= train_df["total_days"]/train_df["listing_id1"]
test_df["diff_rank"]= test_df["total_days"]/test_df["listing_id1"]

# Фича из фото нужно скачать торрент файл
df_time = pd.read_csv('listing_image_time.csv')
train_df = train_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))
test_df = test_df.merge(df_time, how='left', on='listing_id', suffixes=('', '_remove'))

In [3]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price0"] = (train_df["price"]%10).astype(int)
test_df["price0"] = (test_df["price"]%10).astype(int)

In [4]:
# Фичи отсюда https://www.kaggle.com/code/slavik0505/kernel43aae13b62
train_df["price_latitude"] = (train_df["price"])/ (train_df["latitude"]+1.0)
test_df["price_latitude"] =  (test_df["price"])/ (test_df["latitude"]+1.0)

train_df["price_longtitude"] = (train_df["price"])/ (train_df["longitude"]-1.0)
test_df["price_longtitude"] =  (test_df["price"])/ (test_df["longitude"]-1.0)  

In [5]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1                #  скорость обучения
    param['max_depth'] = 6             # Глубина дерева
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 5
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [16]:
import lightgbm as lgb

In [6]:
features_to_use  = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price']

In [7]:
features_to_use.extend(['num_photos', 
                        'num_features', 
                        'num_description_words',
                        'created_year', 
                        'created_month', 
                        'created_day', 
                        'listing_id', 
                        'created_hour', 
                        'clusters',
                        'distance_manh',
                        'distance_central_park',
                        'room_difference',
                        'total_rooms',
                        'price_per_room',
                         'bronx',
                        'distance_central_park',
                        'queens', 
                        'brooklin_cometery', 
                        'brooklin', 
                        'clusters',
                        'num_description_words', 
                        'created_year',
                        'created_month', 
                        'created_day', 
                        'created_hour', 
                        'room_difference',
                        'total_rooms', 
                        'subway_distance', 
                        "density", 
                        "price_latitude", 
                        "price_longtitude", 
                        'total_days', 
                        'diff_rank', 
                        'time_stamp', 
                        'price0', 
                        'price_per_bedroom', 
                        'price_per_bathroom'])

In [11]:
train_df['features'] = train_df['features'].apply(lambda x: ' '.join(["_".join(i.split(' ')) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: ' '.join(["_".join(i.split(' ')) for i in x]))
print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df['features'])
te_sparse = tfidf.transform(test_df['features'])

0    Dining_Room Pre-War Laundry_in_Building Dishwa...
1    Doorman Elevator Laundry_in_Building Dishwashe...
2    Doorman Elevator Laundry_in_Building Laundry_i...
3                                                     
4    Doorman Elevator Fitness_Center Laundry_in_Bui...
Name: features, dtype: object


In [12]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

(49352, 249) (74659, 249)


In [13]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[16:26:33] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mlogloss:1.03163	test-mlogloss:1.03279
[1]	train-mlogloss:0.97588	test-mlogloss:0.97803
[2]	train-mlogloss:0.92644	test-mlogloss:0.92996
[3]	train-mlogloss:0.88370	test-mlogloss:0.88815
[4]	train-mlogloss:0.84720	test-mlogloss:0.85253
[5]	train-mlogloss:0.81420	test-mlogloss:0.82081
[6]	train-mlogloss:0.78544	test-mlogloss:0.79318
[7]	train-mlogloss:0.76011	test-mlogloss:0.76887
[8]	train-mlogloss:0.73769	test-mlogloss:0.74706
[9]	train-mlogloss:0.71799	test-mlogloss:0.72814
[10]	train-mlogloss:0.70036	test-mlogloss:0.71139
[11]	train-mlogloss:0.68424	test-mlogloss:0.69632
[12]	train-mlogloss:0.66972	test-mlogloss:0.68300
[13]	train-mlogloss:0.65697	test-mlogloss:0.67128
[14]	train-mlogloss:0.64516	test-mlogloss:0.66039
[15]	train-mlogloss:0.63453	test-mlogloss:0.65051
[16]	train-mlo

[159]	train-mlogloss:0.40236	test-mlogloss:0.51409
[160]	train-mlogloss:0.40170	test-mlogloss:0.51405
[161]	train-mlogloss:0.40114	test-mlogloss:0.51412
[162]	train-mlogloss:0.40077	test-mlogloss:0.51404
[163]	train-mlogloss:0.39996	test-mlogloss:0.51398
[164]	train-mlogloss:0.39939	test-mlogloss:0.51394
[165]	train-mlogloss:0.39878	test-mlogloss:0.51385
[166]	train-mlogloss:0.39838	test-mlogloss:0.51372
[167]	train-mlogloss:0.39785	test-mlogloss:0.51366
[168]	train-mlogloss:0.39736	test-mlogloss:0.51363
[169]	train-mlogloss:0.39684	test-mlogloss:0.51365
[170]	train-mlogloss:0.39625	test-mlogloss:0.51363
[171]	train-mlogloss:0.39576	test-mlogloss:0.51357
[172]	train-mlogloss:0.39523	test-mlogloss:0.51345
[173]	train-mlogloss:0.39474	test-mlogloss:0.51339
[174]	train-mlogloss:0.39418	test-mlogloss:0.51324
[175]	train-mlogloss:0.39368	test-mlogloss:0.51317
[176]	train-mlogloss:0.39322	test-mlogloss:0.51315
[177]	train-mlogloss:0.39255	test-mlogloss:0.51302
[178]	train-mlogloss:0.39189	te

In [14]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("new_xgboost_new1.csv", index=False)

[16:27:36] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

